# COGS 108 - Data Checkpoint

# Names

- Kaiyuan Yang
- Hainan Xiong
- Keshan Chen
- Jinzong Que

<a id='research_question'></a>
# Research Question

We want to predict whether the collision involves hit and run given various factors, including traffic volumn, areas of San Diego, streetlight locations, streetlight models, streetlight types, the time of incidents happen, collision types, whether the incident involve people injured, and killed. We believe this project will help cops to determine whether they should send more force to handle the collision to prevent potential hit and run. 

# Dataset(s)

*Fill in your dataset information here*


    - Dataset Name:pd_collisions_datasd_v1.csv
    - Link to the dataset:https://data.sandiego.gov/datasets/police-collisions/
    - Number of observations:60461;Number of observations after data cleaning:60456
          This dataset is recorded from San Diego Police Department. This dataset is up-to-date since 2015. It includes variables include report_id, the date and time of incidents happen, the location of incidents happen, the incident types, incident description, number people injured during the accident, number people killed during the accident, whether hit and run. 

    - Dataset Name:streetlight_locations_datasd_v1.csv
    - Link to the dataset:https://data.sandiego.gov/datasets/streetlight_inventory/
    - Number of observations:61229; Number of observations after data cleaning: 36288
          This dataset is recorded from SDPD as well. This dataset includes streetlight ID in SAP, streelight model, streetlight type, longitute, latitude and location of the streetlight.
  
    - Dataset Name:traffic_counts_datasd_v1.csv
    - Link to the dataset:https://data.sandiego.gov/datasets/traffic-volumes/
    - Number of observations:12434; Number of observations after data cleaning: 12270
        This dataset is recorded from SDPD as well. This dataset includes street names, the street limit, count of vehicles from different directions, total counts of vehicles, and the date of each count. 

c

# Setup

In [228]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
collision_data = pd.read_csv('pd_collisions_datasd_v1.csv')
streetlight_data = pd.read_csv('streetlight_locations_datasd_v1.csv')
volumn_data = pd.read_csv('traffic_counts_datasd_v1.csv')

# Data Cleaning

Describe your data cleaning steps here.

In [229]:
# drop the irrelevant columns
temp = collision_data.drop(columns = ['report_id','police_beat','address_pd_intersecting','address_name_intersecting','address_sfx_intersecting','violation_section','violation_type'])

#if hit&run happened, then replace the cell with 1, if not, then replace with 0
temp['hit_run_lvl'] = temp['hit_run_lvl'].replace(('MISDEMEANOR','FENOLY'),(1,1))
temp['hit_run_lvl'].fillna(0,inplace = True)

#if there are people injured or killed, then replace the cell with 1, otherwise replace with 0 
temp['injured'].loc[(temp['injured'] != 0)] = 1
temp['killed'].loc[(temp['killed'] != 0)] = 1

#if the incident happened at night (After 6 p.m. but before 6 a.m.),then replace with 1, 
#if the incident happened at day time (Between 6 a.m. and 6 p.m.),then replace with 0, 
temp['date_time'] = temp['date_time'].str.slice(start = 11,stop = 13)
temp['date_time'] = pd.to_numeric(temp['date_time'])
temp['date_time'].loc[(temp['date_time'] > 18) | (temp['date_time'] < 6 ) ] = 0
temp['date_time'].loc[(temp['date_time'] <= 18) & (temp['date_time'] >= 6 )] = 1

#Create a new address column to combine the street name, stree number and street type
temp['address_no_primary'] = temp['address_no_primary'].apply(str)
temp['address'] =   temp['address_no_primary'] + ' ' + temp['address_pd_primary']+ ' '+temp['address_road_primary'] + ' ' + temp['address_sfx_primary']
temp = temp.drop(columns = ['address_road_primary','address_sfx_primary','address_no_primary','address_pd_primary'])
collision_data = temp
collision_data = collision_data.dropna()
collision_data

,date_time,charge_desc,injured,killed,hit_run_lvl,address
0,0,TURNING MOVEMENTS AND REQUIRED SIGNALS,0,0,1,6000 VISTA SAN ISIDRO
1,0,TURNING MOVEMENTS AND REQUIRED SIGNALS,0,0,1,3700 VERMONT STREET
2,0,TURNING MOVEMENTS AND REQUIRED SIGNALS,1,0,0,100 E SAN YSIDRO BOULEVARD
3,0,MISCELLANEOUS HAZARDOUS VIOLATIONS OF THE VEHI...,0,0,1,2600 RACHEL
4,0,TURNING MOVEMENTS AND REQUIRED SIGNALS,1,0,1,4600 30TH STREET
...,...,...,...,...,...,...
60456,1,FOLLOWING TOO CLOSELY (I),1,0,0,1200 EXPOSITION WAY
60457,1,JAYWALKING (I),1,0,0,1300 01ST AVENUE
60458,1,MISCELLANEOUS HAZARDOUS VIOLATIONS OF THE VEHI...,0,0,0,0 CHAUNCEY STREET
60459,1,SIGNAL LIGHTS:CIRCULAR RED (I),1,0,1,1300 TORREY PINES ROAD


In [230]:
#drop the irrelavant columns and the null value in the traffic volume dataset
temp1 = volumn_data.drop(columns = ['date_count','id','limits','northbound_count','southbound_count','eastbound_count','westbound_count'])
volumn_data = temp1.dropna()
volumn_data 

,street_name,total_count,file_no
0,01 AV,18010.0,0182-07
1,01 AV,20060.0,0152-10
2,01 AV,19597.0,0182-13
3,01 AV,2298.0,0888-12
4,01 AV,15625.0,0687-11
...,...,...,...
12265,ZOO PL,6300.0,0092-05
12266,ZOO PL,9430.0,0137-08
12267,ZOO PL,5595.0,0330-11
12268,ZOO PL,4425.0,1126-14


In [231]:
#drop the irrelavant columns and the null value in the streelight dataset
temp2 = streetlight_data.dropna()
streetlight_data = temp2.drop(columns = ['sap_id','streetlight_wattage','streetlight_voltage'])
streetlight_data

,streetlight_model,streetlight_type,lng,lat,location_description
0,COBRA,IND,-117.252424,32.864282,LA JOLLA SHORES DR @ RUETTE MONTE CARLO N/E
1,COBRA,IND,-117.250892,32.872267,LA JOLLA SHORES LN N/O LA JOLLA SHORES DR @ CDS
2,COBRA,IND,-117.246841,32.881493,BROOKMEAD LN E/O BLACK GOLD RD @ CDS
3,COBRA,IND,-117.248476,32.885452,BLACK GOLD RD E/O LA JOLLA FARMS RD 200' N/S
4,COBRA,IND,-117.247382,32.885273,BLACK GOLD RD E/O LA JOLLA FARMS RD 680' S/S
...,...,...,...,...,...
54706,COBRA,IND,-117.120433,32.763269,34TH ST @ ADAMS AV SE
54709,COBRA,IND,-117.120791,32.763430,34TH ST @ ADAMS AV NW
54710,COBRA,IND,-117.120586,32.763474,34TH ST @ ADAMS AV NE
54711,COBRA,IND,-117.120727,32.763240,34TH ST @ ADAMS AV SW


# Project Proposal (updated)

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM | Read & Think about COGS 108 expectations; brainstorm topics/questions  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/14  | 6 PM  | Import & Wrangle Data (Kaiyuan Yang); EDA (Keshan Chen) | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin Analysis (Jinzong Que; Hainan Xiong) | Discuss/edit Analysis; Complete project check-in |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion (All group members)| Discuss/edit full project |
| 3/19  | Before 11:59 PM  | NA | Turn in Final Project & Group Project Surveys |